# General Explanation:

In the present jupyer notebook, we will desglose the work for characterizing how entanglement is relevant in the anastz embedding and in the finding of the ground state for quantum chemistry problems.

Preeliminaries:

In Variational Quantum Algorithms (VQA), It is usual to define a cost function, given a Quantum State $\rho$ and an Observable $\mathcal{O}$ such that:

$$
C(\rho, \mathcal{O}) = \text{Tr}(\rho \mathcal{O})
$$
and then minimize it or optimize it in order to tackle down optimization Problems.

An issue with this method, is that often exploring the whole landscape of states it's an exponential task. (the so-called curse of dimensionality)

Several methods have been developed utilizing the stabilizer formalism, correct walk at the Hilbert Space, and so on.

In this case, we are going to explore how different classes of entanglement can contribute to find the ground state of a molecular Hamiltonian.

For this case, we now that a Molecular Hamiltonian can be expresed by the electronic structure Hamiltonian, $\mathcal{H}_{\text{ES}}$:

$$
\mathcal{H}_{\text{ES}} =  T_{N} + T_{e} + V_{ee}^{\text{Coulomb}} + V_{en}^{\text{Coulomb}} + V_{nn}^{\text{Coulomb}}
$$

Where $T_{N}, T_{e} $ are the kinetic energy of the Nuclei and electrons, respectively, while $V_{ee}^{\text{Coulomb}}, V_{en}^{\text{Coulomb}}, V_{nn}^{\text{Coulomb}}$ are the coulomb interactions between electron-electron, electron-nuclei and nuclei-nuclei respectively.

In this case, by utilizing the B-O approx, we are going only to center on the 2nd, 3rd and 4th term. (which is a valid approximation because we want to focuse on the low-energy regime)

In order to avoid the problem of barren plateau and correct ansatz preparation, we are going to characterize how well does different states of different classes computes the GS for 4-qubit problems.

The idea is to reach all the Hilbert Space in these sub division of states, and the utilizing later to tackle down more efficiently these problems.



# Beggining:

first, we are going to install the necesary packages:

In [8]:
%pip install qutip
%pip install pyscf
%pip install qiskit_nature


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 25.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached qiskit_nature-0.7.2-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 3.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 3.3 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.3-py3-none-any.whl (6.2 

## Operator creation:

In this part, we are going to construc the Unitary operators that are going to be layered in the final class.

In [2]:
#We import the necesary libraries:
import qutip as qt
from qutip import Bloch, QobjEvo, basis, sesolve, sigmay, sigmaz
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
from scipy.constants import h, e

# Operadores de Pauli en el qubit k
def SX(N, k):
    """Create a sigma_X operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N 
    L[k] = qt.sigmax()
    return qt.tensor(L)

def SY(N, k):
    """Create a sigma_Y operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N
    L[k] = qt.sigmay()
    return qt.tensor(L)

def SZ(N, k):
    """Create a sigma_Z operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N
    L[k] = qt.sigmaz()
    return qt.tensor(L)


from qutip import create, destroy

def Sm(N, k):
    """Create a sigma_X operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N 
    L[k] = destroy(2)
    return qt.tensor(L)

def Sp(N, k):
    """Create a sigma_X operator at position k from
    a Hilbert space of N qubits
    """
    L = [qt.qeye(2)]*N 
    L[k] = create(2)
    return qt.tensor(L)



In [4]:
def unitary(theta: float, phi: float, lamb) -> np.ndarray:
    c = np.cos(theta / 2)
    s = np.sin(theta / 2)
    e_phi = np.exp(1j * phi)
    e_lamb = np.exp(1j * lamb)

    return np.array([[c,                - e_lamb * s],
                     [-e_phi * s,    e_lamb * e_phi * c  ]])



def U_qubit(N : int, k : int, params : list ):
    
    """
    N: number of total qubits
    k: label of the target qubit to apply the local unitary U
    params: list or array where it contains the parameters that characterize the unitary
    """

    # we define the matrix 
    U = unitary(*params)

    # Converts from numpy array to Qobj of Qutip
    U_qobj = qt.Qobj(U)

    # We construct the tensorized operator in qutip
    L = [qt.qeye(2)] * N  # Lista con Identidades en todos los sitios
    L[k] = U_qobj  # Sustituye en el qubit k
    return qt.tensor(L)




In [7]:
# Num of qubits
N = 4  

# Vector de parámetros (12 elementos en total: 3 por cada qubit)

#question: why these values?
#can this affect the convergence time?
params_vec = [np.pi/4, np.pi/3, np.pi/6,  # Qubit 0
              np.pi/2, np.pi/4, np.pi/8,  # Qubit 1
              np.pi/3, np.pi/6, np.pi/2,  # Qubit 2
              np.pi/8, np.pi/3, np.pi/4]  # Qubit 3

params_vecV2 = np.random.uniform(0, 2*np.pi, 12)

# Now we generate the unitary operators for each qubit

dict_op = {f'U_{i}': U_qubit(N, i, params_vec[3*i: 3*(i+1)]) for i in range(N)}


# Comprobamos los operadores generados
#print(dict_op['U_1'])
#...

In this part, we are going to construct The operator $\mathcal{H}_{\text{ES}}$. In order to do this we are going to utilize the PySCF package together with Qiskit Nature package:

With this form, we can express the hamiltonian as a QuTIP object.

In [22]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

def H2_Hamiltonian_Operator(r_i):
    driver = PySCFDriver(
        atom= f'H .0 .0 .0; H .0 .0 {r_i}',
        unit = DistanceUnit.ANGSTROM,
        basis= 'sto-3g',
    )

    problem = driver.run()
    second_q_ham = problem.hamiltonian.second_q_op()

    mapper = JordanWignerMapper()

    pauli_hamiltonian_op = mapper.map(second_q_ham)
    matrix_representation = pauli_hamiltonian_op.to_matrix()
    qt_hamiltonian_op = qt.Qobj(matrix_representation)
    
    return matrix_representation


In [17]:

# Función de coste para el VQE simulado


def Cost(params, *args):
    U_0 = U_qubit(N, 0, params[0:3])  # U en el qubit 0
    U_1 = U_qubit(N, 1, params[3:6])  # U en el qubit 1
    U_2 = U_qubit(N, 2, params[6:9])  # U en el qubit 2
    U_3 = U_qubit(N, 3, params[9:12])
    psi_e = args[0]
    H = args[1]

    Psi = U_3*U_2*U_1*U_0*psi_e
    
    cost = qt.expect(H, Psi)
    return cost
    

In [19]:
N = 4
s0 = qt.basis(2, 0)
s1 = qt.basis(2, 1)

#ESTADOS DE CADA CLASE DE ENTRELAZAMIENTO
psi_C9 = 1/2*(qt.tensor(s0,s0,s0,s0)+qt.tensor(s0,s1,s0,s1)+qt.tensor(s1,s0,s1,s0) + qt.tensor(s1,s1,s1,s1) )
psi_C3 = 1/np.sqrt(2)*(qt.tensor(s0,s0,s0,s0)+ qt.tensor(s1,s0,s1,s0))
psi_C1 = qt.tensor(s0,s0,s0,s0)
psi_C11 = 1/np.sqrt(3)*(qt.tensor(s0,s0,s1,s0)+qt.tensor(s0,s1,s0,s0)+qt.tensor(s1,s0,s0,s0))
psi_C15 = 1/np.sqrt(2)*(qt.tensor(s0,s0,s0,s0)+ qt.tensor(s1,s1,s1,s0))
psi_C19 = 1/2*(qt.tensor(s1,s0,s0,s0)+qt.tensor(s0,s1,s0,s0)+qt.tensor(s0,s0,s1,s0)+ qt.tensor(s0,s0,s0,s1))
psi_C26 = 1/np.sqrt(2)*(qt.tensor(s0,s0,s0,s0)+qt.tensor(s1,s1,s1,s1))
psi_C48 = 1/np.sqrt(8)*(qt.tensor(s0,s0,s0,s0)+qt.tensor(s0,s0,s1,s1)+qt.tensor(s0,s1,s0,s1)+qt.tensor(s0,s1,s1,s0)+qt.tensor(s1,s0,s0,s0)-qt.tensor(s1,s0,s1,s1)+qt.tensor(s1,s1,s0,s1)-qt.tensor(s1,s1,s1,s0))

# Hamiltoniano
def construct_hamiltonian(N, Ops, coeffs):
    """
    Construct the Hamiltonian given the operators and coefficients.

    Args:
        N: Number of qubits in the system.
        Ops: List of operator strings (e.g., 'IIIZ', 'XXYY').
        coeffs: List of coefficients corresponding to the operators.

    Returns:
        Hamiltonian as a qutip.Qobj object.
    """
    I = np.eye(2**(N))
    I = qt.Qobj(I) #Identity in the full Hilbert space
    I.dims = [[2]*(N), [2]*(N)]
    H = 0 * I  # Initialize Hamiltonian as zero operator

    for op, coeff in zip(Ops, coeffs):
        term = I  # Start with identity operator

        for i, char in enumerate(op):
            if char == 'X':
                term = term * SX(N, i)
            elif char == 'Y':
                term = term * SY(N, i)
            elif char == 'Z':
                term = term * SZ(N, i)

        H += coeff * term  # Add the term with its coefficient

    return H
# Convertimos a elementos para qutip sacados de qiskit. Mapeo: Jordan Wigner

#Molecula H2 a r = 0.741
Ops = ['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'] #aqui se cambia

# Extraer los coeficientes
coeffs = [-0.09835117+0.j,  0.17125916+0.j, -0.22297019+0.j,  0.17125916+0.j,
 -0.22297019+0.j,  0.12056779+0.j,  0.16864127+0.j,  0.1658844 +0.j,
  0.0453166 +0.j,  0.0453166 +0.j,  0.0453166 +0.j,  0.0453166 +0.j,
  0.1658844 +0.j,  0.17436685+0.j,  0.12056779+0.j] #aqui igual

# Hamiltoniano para el VQE simulado
Hf = construct_hamiltonian(N, Ops, coeffs)

In [24]:
import numpy as np
from scipy.optimize import minimize

# Definir los límites de los parámetros manualmente para COBYLA
constraints = []
for i in range(12):
    lower_bound = 0
    upper_bound = np.pi if i % 3 == 0 else 2 * np.pi  # Cada tercer parámetro tiene límite en pi
    constraints.append({'type': 'ineq', 'fun': lambda x, i=i: x[i] - lower_bound})  # x[i] >= lower_bound
    constraints.append({'type': 'ineq', 'fun': lambda x, i=i: upper_bound - x[i]})  # x[i] <= upper_bound

# Inicialización de los parámetros aleatorios dentro de los límites
x = [np.random.uniform(0, np.pi) if i % 3 == 0 else np.random.uniform(0, 2*np.pi) for i in range(12)]

# Argumentos de la función de costo
args1 = (psi_C3, Hf) #(Estado de la clase de entrelazamiento, Hamiltoniano de la molecula)

# Minimización usando COBYLA
VQE = minimize(Cost, x, args=args1, method='COBYLA', constraints=constraints, options={'maxiter': 1500, 'disp': True})

print(VQE)


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.45511627629656654
       x: [ 1.937e+00  2.635e+00  4.513e+00  3.784e+00  5.988e+00
            9.105e-01  1.205e+00  5.276e+00  4.911e+00  2.498e+00
            2.345e+00  1.325e-01]
    nfev: 481
   maxcv: 0.0


In [ ]:

def Cost(params, *args):
    U_0 = U_qubit(N, 0, params[0:3])  # U en el qubit 0
    U_1 = U_qubit(N, 1, params[3:6])  # U en el qubit 1
    U_2 = U_qubit(N, 2, params[6:9])  # U en el qubit 2
    U_3 = U_qubit(N, 3, params[9:12])
    psi_e = args[0]
    H = args[1]

    Psi = U_3*U_2*U_1*U_0*psi_e
    
    cost = qt.expect(H, Psi)
    return cost